In [154]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
%matplotlib inline

# Reading in & combining the data

In [155]:
# Creating a function to combine the datasets
def df_combiner(data_folder_path):
    '''
    This function combines datasets in a folder by row, assuming that each dataset has the same columns.
    '''
    # Getting paths of each dataset in folder
    d = data_folder_path
    data_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d, path)
        if os.path.isfile(full_path):
            data_paths.append(full_path)

    # Joining the data together by row
    big_df = []
    for path in data_paths:
        df = pd.read_csv(path, parse_dates = True)
        big_df.append(df)
    df = pd.concat(big_df, ignore_index=True)
    return df

data_folder = '/Users/logno/Documents/GitHub/tech-sales-analysis/Sales_Data'
df = df_combiner(data_folder)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...
186845,222905,AAA Batteries (4-pack),1,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
186846,222906,27in FHD Monitor,1,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
186847,222907,USB-C Charging Cable,1,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
186848,222908,USB-C Charging Cable,1,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


# Checking for missing values

In [156]:
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [157]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

In [158]:
df.loc[df['Order ID'].isnull()]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
264,NaN,NaN,NaN,NaN,NaN,NaN
648,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,NaN,NaN,NaN
1385,NaN,NaN,NaN,NaN,NaN,NaN
1495,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
185795,NaN,NaN,NaN,NaN,NaN,NaN
185868,NaN,NaN,NaN,NaN,NaN,NaN
185887,NaN,NaN,NaN,NaN,NaN,NaN
185960,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
df.shape

(186850, 6)

It looks like the data is missing 545 rows of data. I think that it is safe to drop this data.

In [160]:
df = df.loc[df['Order ID'].notna()]

In [161]:
df.shape

(186305, 6)

# Changing data types for numeric columns

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186305 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


The columns that need attention are Order ID, Quantity Ordered, Price Each, and Order Date.

In [163]:
# Dropping Order ID because I don't think it will help much in the analysis
df = df.drop(['Order ID'], axis=1)
df.columns

Index(['Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

In [164]:
# Converting Quantity Ordered column to numeric. If there are any objects that cannot be converted, they become null values.
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors = 'coerce')

In [165]:
# Looking at how many new null values in this column
df['Quantity Ordered'].isnull().sum()

355

In [166]:
# Finding the problem rows with null values
bad_rows = df.loc[df['Quantity Ordered'].isnull()]
bad_rows

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
254,Product,NaN,Price Each,Order Date,Purchase Address
705,Product,NaN,Price Each,Order Date,Purchase Address
1101,Product,NaN,Price Each,Order Date,Purchase Address
2875,Product,NaN,Price Each,Order Date,Purchase Address
3708,Product,NaN,Price Each,Order Date,Purchase Address
...,...,...,...,...,...
183671,Product,NaN,Price Each,Order Date,Purchase Address
184012,Product,NaN,Price Each,Order Date,Purchase Address
184041,Product,NaN,Price Each,Order Date,Purchase Address
184275,Product,NaN,Price Each,Order Date,Purchase Address


It seems these 355 rows are no good. They have the column name printed in each row, making them unusable. I will drop these rows.

In [167]:
# Dropping problem rows
bad_rows_index = [index for index in bad_rows.index]
df = df.drop(bad_rows_index, axis = 0)
df

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,Macbook Pro Laptop,1.0,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,LG Washing Machine,1.0,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,USB-C Charging Cable,1.0,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,27in FHD Monitor,1.0,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,USB-C Charging Cable,1.0,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...
186845,AAA Batteries (4-pack),1.0,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
186846,27in FHD Monitor,1.0,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
186847,USB-C Charging Cable,1.0,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
186848,USB-C Charging Cable,1.0,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


In [168]:
# Converting Price Each column to numeric
df['Price Each'] = pd.to_numeric(df['Price Each'])

In [169]:
df.dtypes

Product              object
Quantity Ordered    float64
Price Each          float64
Order Date           object
Purchase Address     object
dtype: object

In [170]:
# Converting order date column into date dtype
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [171]:
df.dtypes

Product                     object
Quantity Ordered           float64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
dtype: object

# Adding a columns for order times (month, day, hour)

In [172]:
df.head()

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,Macbook Pro Laptop,1.0,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001"
1,LG Washing Machine,1.0,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001"
2,USB-C Charging Cable,1.0,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001"
3,27in FHD Monitor,1.0,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016"
4,USB-C Charging Cable,1.0,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301"


In [173]:
df['Order_Month'] = pd.DatetimeIndex(df['Order Date']).month
df.head()

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Order_Month
0,Macbook Pro Laptop,1.0,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12
1,LG Washing Machine,1.0,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12
2,USB-C Charging Cable,1.0,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12
3,27in FHD Monitor,1.0,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12
4,USB-C Charging Cable,1.0,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12


In [174]:
df['Order_Hour'] = pd.DatetimeIndex(df['Order Date']).hour
df.head()

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Order_Month,Order_Hour
0,Macbook Pro Laptop,1.0,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,0
1,LG Washing Machine,1.0,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,7
2,USB-C Charging Cable,1.0,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,18
3,27in FHD Monitor,1.0,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,15
4,USB-C Charging Cable,1.0,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12


In [175]:
df['Order_Day'] = pd.DatetimeIndex(df['Order Date']).day
df.head()

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Order_Month,Order_Hour,Order_Day
0,Macbook Pro Laptop,1.0,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,0,30
1,LG Washing Machine,1.0,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,7,29
2,USB-C Charging Cable,1.0,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,18,12
3,27in FHD Monitor,1.0,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,15,22
4,USB-C Charging Cable,1.0,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12,18


# Adding a sale value column

In [176]:
df['Sale_Value'] = df['Quantity Ordered'] * df['Price Each']
df.head()

,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Order_Month,Order_Hour,Order_Day,Sale_Value
0,Macbook Pro Laptop,1.0,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,0,30,1700.00
1,LG Washing Machine,1.0,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,7,29,600.00
2,USB-C Charging Cable,1.0,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,18,12,11.95
3,27in FHD Monitor,1.0,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,15,22,149.99
4,USB-C Charging Cable,1.0,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12,18,11.95


In [177]:
df.dtypes

Product                     object
Quantity Ordered           float64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
Order_Month                  int64
Order_Hour                   int64
Order_Day                    int64
Sale_Value                 float64
dtype: object